In [1]:
import pandas as pd
import pyodbc
import os
import datetime as dt
import shutil

In [2]:
# retrieve shortage qty from SQL, get unique list of FV/ODM in the past 6 months
# compare with the FV/ODM list on the very day, get the difference list
# amend qty 0 for the different list

In [3]:
home = os.path.expanduser('~')
today = dt.datetime.today().date()
# retrieve date of 180 days ago
target_date =str(today - dt.timedelta(days = 180))

In [4]:
# # connect SQL server
conn = pyodbc.connect('Driver={SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# query statement
# select shortage data for the past 6 months with shortage qty not equal to zero
Query = 'SELECT [ODM],[Item],[Commodity],[FV],[Platform],[HP_PN],[BuyerName] FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate >= ? AND [Total Shortage Qty] != 0'
params = (target_date)
cursor.execute(Query, params)
shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
cursor.close()
conn.close()

In [5]:
print(len(shortage))

27078


In [6]:
shortage['len'] = shortage['HP_PN'].apply(lambda x: len(x))
shortage.sort_values(by = 'len', inplace = True, ascending=True)
shortage.drop(columns = 'len', inplace = True)
shortage = shortage.drop_duplicates(keep = 'last')
print(len(shortage))

9148


In [7]:
shortage = shortage.drop_duplicates(subset = ['ODM','Commodity','FV'],keep = 'last')
print(len(shortage))
shortage_tmp = shortage[['ODM','Commodity','FV']]

1303


In [8]:
today_shortage = pd.read_excel(os.path.join(home, 'Desktop', 'shortage_all.xlsx'), sheet_name='Sheet1' )
today_shortage = today_shortage[['ODM','Commodity','FV']]

In [9]:
merge = shortage_tmp.merge(today_shortage, on = ['ODM','Commodity','FV'], how = 'outer', indicator= True)

In [10]:
print(len(merge[merge['_merge'] == 'left_only']))
print(len(merge[merge['_merge'] == 'right_only']))
print(len(merge[merge['_merge'] == 'both']))

1058
30
278


In [11]:
amend_data = merge[merge['_merge'] == 'left_only'].drop(columns = '_merge').reset_index(drop = True)
amend_data = amend_data.merge(shortage, on = ['ODM','Commodity','FV'], how = 'left')
# amend_data['Total Shortage Qty'] = 0
amend_data['ReportDate'] = today
amend_data['last FD date'] = today
amend_list = ['P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside']
for _ in amend_list:
    amend_data[_] = 0


In [12]:
amend_data.head()

,ODM,Commodity,FV,Item,Platform,HP_PN,BuyerName,ReportDate,last FD date,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside
0,CEI,AC ADAPTOR,100W Mario II tube USBC,ADA77,Regency 24C1,"N56858-011,N56858-012,N56858-013",ChenJosephine(GPS),2024-10-30,2024-10-30,0,0,0,0,0,0
1,CEI,AC ADAPTOR,100W Mario tube USBC,ADA76,Sparty 24C1/Canopy 24C1,"N56858-003,N56858-002,N56858-001",FeiFion(GPSShanghai),2024-10-30,2024-10-30,0,0,0,0,0,0
2,CEI,AC ADAPTOR,230W Rhea II tube,ADA73,Thetiger 23C1/Roaree 23C1/Roaree 24C1/Brunobea...,"M85389-001,M85389-003",HsiehAngela,2024-10-30,2024-10-30,0,0,0,0,0,0
3,CEI,AC ADAPTOR,65W EM II,ADA80,MaldivesR6u 2.0,"L25299-012,L25299-011",FeiFion(GPSShanghai),2024-10-30,2024-10-30,0,0,0,0,0,0
4,CEI,AC ADAPTOR,65W EM plus,ADA47,Maldives R6U/Herschel R6U,"L25299-001,L25299-002",ChenJosephine(GPS),2024-10-30,2024-10-30,0,0,0,0,0,0


In [13]:
amend_data.to_excel(os.path.join(home, 'amend_stge_zero.xlsx'))

In [14]:
# upload amend data to SQL
conn = pyodbc.connect('Driver={SQL Server}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

for index, row in amend_data.iterrows():
    s_ODM = row['ODM']
    s_Item = row['Item']
    s_Commodity = row['Commodity']
    s_FV = row['FV']
    s_Platform = row['Platform']
    s_P1 = row['P1']
    s_P2 = row['Net P2']
    s_P3 = row['Net P3']
    s_Total = row['Total Shortage Qty']
    s_BT = row['BT shortage']
    s_working = row['Working on upside']
    s_ReportDate = pd.to_datetime(row['ReportDate'])
    s_lastFDdate = pd.to_datetime(row['last FD date'])
    s_BuyerName = row['BuyerName']
    s_PN = row['HP_PN']
    try:
        cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_shortage_zero ( ODM,Item,Commodity,FV,Platform,P1,[Net P2],[Net P3],[Total Shortage Qty],[BT shortage],[Working on upside],ReportDate,[last FD date],HP_PN,BuyerName )\
                        VALUES('{s_ODM}','{s_Item}','{s_Commodity}','{s_FV}','{s_Platform}','{s_P1}','{s_P2}','{s_P3}','{s_Total}','{s_BT}','{s_working}','{s_ReportDate}','{s_lastFDdate}','{s_PN}','{s_BuyerName}')".replace("'NaT'", "NULL"))
    except:
        print(f'Error on {s_ODM},{s_Item},{s_Commodity},{s_FV},{s_Platform},{s_P1},{s_P2},{s_P3},{s_Total},{s_BT},{s_working},{s_ReportDate},{s_lastFDdate},{s_PN},{s_BuyerName}')

conn.commit()
conn.close()

Error on CQIEC,1,ODD,ODD DVD SATA 9.5mm Slim Tray,BOGGLE/BoggleR,0,0,0,0,0,0,2024-10-30 00:00:00,2024-10-30 00:00:00,849054-6C7,ManeeratKosit(Kosit-GPSThailand)
Error on CQIEC,1.0,ODD,ODD DVDSM 9.5mm Slim Fixed,Sleeves,0,0,0,0,0,0,2024-10-30 00:00:00,2024-10-30 00:00:00,762432-800,849055-6F7,ManeeratKosit(Kosit-GPSThailand)
Error on CQWIS,1.0,HDD,2TB 7200 SATA 3.5,ArticunoI ArcticOCR/Snowball,0,0,0,0,0,0,2024-10-30 00:00:00,2024-10-30 00:00:00,661702-002,M50324-001,N20653-001,JittungmanSupansa (Suzii - GPS TH)
Error on WHFXN,7,ODD,ODD DVD-Writer 9.5mm Ultra Slim Tray,Kombucha/AdlerI/WatsonI/Jenga,0,0,0,0,0,0,2024-10-30 00:00:00,2024-10-30 00:00:00,849055-6C7,ManeeratKosit(Kosit-GPSThailand)


In [15]:
# 超過180以前等於0的要刪掉，不然會無限長大
# delte shortage = 0 where report date is before 180 days, otherwise, the database will grow up without limit

In [16]:
# # # connect SQL server
# conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
# cursor = conn.cursor()
# # query statement
# Query = 'DELETE FROM OPS.GPS_tbl_ops_shortage_ext WHERE ReportDate <= ? AND [Total Shortage Qty] = 0'
# params = (target_date)
# cursor.execute(Query, params)
# shortage = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# # shortage.columns = ['ODM','Item','Commodity','FV','Platform','P1','Net P2','Net P3','Total Shortage Qty','BT shortage','Working on upside','ReportDate','last FD date','HP_PN','BuyerName']
# cursor.close()
# conn.close()